In [1]:

import torch
import os
import random
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, TensorDataset

import librosa
import librosa.display
import numba.decorators
import numpy as np
import matplotlib.pyplot as plt
from numba.decorators import jit as optional_jit


from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/content/gdrive')


#PATH = 'C://Projects//keras_talk//keras//intern//dataset//'
PATH = '/content/gdrive/My Drive/dataset/'

train_size = 800
test_size = 200

EPOCHS = 10
BATCH_SIZE = 40


def Y_DATA(y_data):
    for idx in range(y_data.shape[0]):
        y = y_data[idx]
        if y < 0:  y_data[idx] = 10
        else:      y_data[idx] = (y//20)
    return y_data

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


###### data normalization

In [2]:

import numpy as np


dataset_dict = { 0 : 'S_left',        1 : 'S_left_phase',
                 2 : 'S_right',       3 : 'S_right_phase',
                 4 : 'clean_left',    5 : 'clean_left_phase',
                 6 : 'clean_right',   7 : 'clean_right_phase',
                 8 : 'idx_drone_end', 9 : 'idx_voice_end',
                10 : 'idx_voice_start'}


x_data_list = [0,2,1,3]


numpy_dict = dict()
for n in x_data_list:
    numpy_name    = dataset_dict[n]
    numpy_dict[n] = np.load( PATH + numpy_name + '.npy' )
    


'''    x_data,       y_data '''
'''(1000,4,257,382), (1000,)'''

x_data = []
for idx in range(1000):
    x_element = []

    for n in x_data_list:
        x_element.append( numpy_dict[n][:,:,idx] )

    x_element = np.asarray( x_element )
    
    
    # log scale 변환 [dB]
    x_element[0] = 20*np.log10( np.abs(x_element[0]) + np.finfo(np.float32).eps )
    x_element[1] = 20*np.log10( np.abs(x_element[1]) + np.finfo(np.float32).eps )


    #magnitude normalization
    x_mean = x_element[0].mean()
    x_stdv = x_element[0].std()
    x_element[0] = ( x_element[0] - x_mean ) / x_stdv

    x_mean = x_element[1].mean()
    x_stdv = x_element[1].std()
    x_element[1] = ( x_element[1] - x_mean ) / x_stdv


    #phase normalization
    x_mean = x_element[0].mean()
    x_stdv = x_element[0].std()
    x_element[2] = ( x_element[0] - x_mean ) / x_stdv

    x_mean = x_element[1].mean()
    x_stdv = x_element[1].std()
    x_element[3] = ( x_element[1] - x_mean ) / x_stdv


    x_data.append( x_element )


x_data = np.asarray(x_data)
y_data = Y_DATA( np.load(PATH + 'angle.npy') )
print('done..')

done..


In [3]:

#x_data = x_data.reshape()
#y_data = y_data.reshape()



x_data = torch.from_numpy( x_data ).float().to('cuda')
y_data = torch.from_numpy( y_data ).long().to('cuda')

full_dataset = TensorDataset( x_data, y_data )


train_dataset, valid_dataset = torch.utils.data.random_split( full_dataset, [train_size, test_size])
train_dataset = DataLoader( dataset=train_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last=True)
valid_dataset = DataLoader( dataset=valid_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last=True)


#### DenseNet

In [4]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
#import torch.utils.checkpoint as cp
from collections import OrderedDict
from torch import Tensor
#from torch.jit.annotations import List



In [5]:
class _DenseLayer(nn.Module):
    def __init__(self, num_input_features, growth_rate, bn_size, drop_rate, memory_efficient=False):
        super(_DenseLayer, self).__init__()
        
        self.add_module('norm1', nn.BatchNorm2d(num_input_features)),
        self.add_module('relu1', nn.ReLU(inplace=True)),
        self.add_module('conv1', nn.Conv2d(num_input_features, bn_size * growth_rate,
                                           kernel_size=1, stride=1,bias=False)),
        
        self.add_module('norm2', nn.BatchNorm2d(bn_size * growth_rate)),
        self.add_module('relu2', nn.ReLU(inplace=True)),
        self.add_module('conv2', nn.Conv2d(bn_size * growth_rate, growth_rate,
                                           kernel_size=3, stride=1, padding=1,
                                           bias=False)),
        
        self.drop_rate = float(drop_rate)
        self.memory_efficient = memory_efficient

    def bn_function(self, inputs):
        # type: (List[Tensor]) -> Tensor
        concated_features = torch.cat(inputs, 1)
        bottleneck_output = self.conv1(self.relu1(self.norm1(concated_features)))  # noqa: T484
        
        return bottleneck_output


    def forward(self, input):  # noqa: F811
        if isinstance(input, Tensor):
            prev_features = [input]
        else:
            prev_features = input


        bottleneck_output = self.bn_function(prev_features)
        new_features = self.conv2(self.relu2(self.norm2(bottleneck_output)))
        
        if self.drop_rate > 0:
            new_features = F.dropout(new_features, p=self.drop_rate,
                                     training=self.training)
        
        return new_features



In [6]:

class _DenseBlock(nn.ModuleDict):
    _version = 2

    def __init__(self, num_layers, num_input_features, bn_size, growth_rate, drop_rate, memory_efficient=False):
        super(_DenseBlock, self).__init__()
        for i in range(num_layers):
            layer = _DenseLayer(
                num_input_features + i * growth_rate,
                growth_rate=growth_rate,
                bn_size=bn_size,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient,
            )
            self.add_module('denselayer%d' % (i + 1), layer)

    def forward(self, init_features):
        features = [init_features]
        for name, layer in self.items():
            new_features = layer(features)
            features.append(new_features)
        return torch.cat(features, 1)



In [7]:


class _Transition(nn.Sequential):
    def __init__(self, num_input_features, num_output_features):
        super(_Transition, self).__init__()
        self.add_module('norm', nn.BatchNorm2d(num_input_features))
        self.add_module('relu', nn.ReLU(inplace=True))
        self.add_module('conv', nn.Conv2d(num_input_features,
                                          num_output_features,
                                          kernel_size=1, stride=1, bias=False))
        self.add_module('pool', nn.AvgPool2d(kernel_size=2, stride=2))




In [8]:


class DenseNet(nn.Module):
    '''growth_rate, drop_rate'''
    def __init__(self, growth_rate=20, block_config=(6, 12, 24, 16),
                 num_init_features=128, bn_size=4, drop_rate=0.10,
                 num_classes=11, memory_efficient=False):

        super(DenseNet, self).__init__()

        # First convolution
        self.features = nn.Sequential(OrderedDict([
            ('conv0', nn.Conv2d(4, num_init_features, kernel_size=7, stride=2, padding=3, bias=False)),
            ('norm0', nn.BatchNorm2d(num_init_features)),
            ('relu0', nn.ReLU(inplace=True)),
            ('pool0', nn.MaxPool2d(kernel_size=3, stride=2, padding=1)),
        ]))


        # Each denseblock
        num_features = num_init_features
        for i, num_layers in enumerate(block_config):
            block = _DenseBlock(
                num_layers=num_layers,
                num_input_features=num_features,
                bn_size=bn_size,
                growth_rate=growth_rate,
                drop_rate=drop_rate,
                memory_efficient=memory_efficient
            )
            self.features.add_module('denseblock%d' % (i + 1), block)
            num_features = num_features + num_layers * growth_rate

            if i != len(block_config) - 1:
                trans = _Transition(num_input_features=num_features,
                                    num_output_features=num_features // 2)
                self.features.add_module('transition%d' % (i + 1), trans)
                num_features = num_features // 2

        # Final batch norm
        self.features.add_module('norm5', nn.BatchNorm2d(num_features))

        # Linear layer
        self.classifier = nn.Linear(num_features, num_classes)

        # Official init from torch repo.
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.constant_(m.bias, 0)



    def forward(self, x):
        features = self.features(x)
        out = F.relu(features, inplace=True)
        out = F.adaptive_avg_pool2d(out, (1, 1))
        out = torch.flatten(out, 1)
        out = self.classifier(out)
        return out





In [9]:



def _densenet(arch, growth_rate, block_config, num_init_features, pretrained, progress, **kwargs):
    return DenseNet(growth_rate, block_config, num_init_features, **kwargs)



def densenet_custom(pretrained=False, progress=True, **kwargs):
    return _densenet('densenet_custom', 20, (8, 8, 4), 32, pretrained, progress, **kwargs)


##### model train

In [10]:
torch.manual_seed(1)


model = densenet_custom().to('cuda')

criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = torch.optim.Adagrad(model.parameters(), lr=0.00001, weight_decay=0.9)


train_loss = []
train_acc  = []


model.train()
for epoch in range(30):
    print('epoch' + str(epoch+1))
    
    for i, (data, label) in enumerate(train_dataset):
        (data, label) = (data.to('cuda'), label.to('cuda'))

        optimizer.zero_grad()
        output = model(data)
  
        loss = F.nll_loss(output, label.reshape(BATCH_SIZE))
        loss.backward()
        optimizer.step()
        
        preds = output.data.max(1)[1]
        corr  = (preds==label.reshape(BATCH_SIZE)).sum().item()
        acc   = corr/BATCH_SIZE*100
        
        train_loss.append(loss.item())
        train_acc.append( acc )
        
        print('\tLoss: {:.3f}\tAcc: {:.3f}'.format(loss.item(), acc))
        

epoch1
	Loss: -0.078	Acc: 12.500
	Loss: -0.080	Acc: 7.500
	Loss: -0.034	Acc: 12.500
	Loss: -0.122	Acc: 20.000
	Loss: -0.100	Acc: 5.000
	Loss: 0.014	Acc: 0.000
	Loss: -0.029	Acc: 7.500
	Loss: 0.025	Acc: 2.500
	Loss: -0.071	Acc: 10.000
	Loss: -0.018	Acc: 7.500
	Loss: -0.064	Acc: 2.500
	Loss: -0.091	Acc: 5.000
	Loss: -0.037	Acc: 7.500
	Loss: -0.122	Acc: 15.000
	Loss: -0.028	Acc: 2.500
	Loss: -0.032	Acc: 5.000
	Loss: -0.091	Acc: 10.000
	Loss: -0.052	Acc: 15.000
	Loss: -0.120	Acc: 10.000
	Loss: -0.100	Acc: 12.500
epoch2
	Loss: -0.071	Acc: 10.000
	Loss: -0.029	Acc: 2.500
	Loss: -0.092	Acc: 10.000
	Loss: -0.067	Acc: 2.500
	Loss: -0.083	Acc: 12.500
	Loss: -0.038	Acc: 5.000
	Loss: -0.071	Acc: 10.000
	Loss: -0.139	Acc: 22.500
	Loss: -0.019	Acc: 5.000
	Loss: -0.050	Acc: 7.500
	Loss: -0.093	Acc: 7.500
	Loss: -0.111	Acc: 10.000
	Loss: -0.070	Acc: 5.000
	Loss: -0.035	Acc: 5.000
	Loss: -0.059	Acc: 0.000
	Loss: -0.041	Acc: 7.500
	Loss: -0.024	Acc: 15.000
	Loss: -0.123	Acc: 15.000
	Loss: -0.047	Acc: 5.

In [11]:
model

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(4, 32, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(32, 80, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(80, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(80, 20, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(52, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1):